## Imports ##

In [ ]:
import nltk
from nltk.corpus import words
import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

## Data Preparation ##

In [ ]:
nltk.download("words")

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
data = pd.read_csv("kaggle_RC_2019-05.csv")
data.head()

,subreddit,body,controversiality,score
0,gameofthrones,Your submission has been automatically removed...,0,1
1,aww,"Dont squeeze her with you massive hand, you me...",0,19
2,gaming,It's pretty well known and it was a paid produ...,0,3
3,news,You know we have laws against that currently c...,0,10
4,politics,"Yes, there is a difference between gentle supp...",0,1


In [ ]:
# remove controversial statements
data = data[data["controversiality"] != 1]
#data = data[["body"]]
print(data.head())

       subreddit                                               body  \
0  gameofthrones  Your submission has been automatically removed...   
1            aww  Dont squeeze her with you massive hand, you me...   
2         gaming  It's pretty well known and it was a paid produ...   
3           news  You know we have laws against that currently c...   
4       politics  Yes, there is a difference between gentle supp...   

   controversiality  score  
0                 0      1  
1                 0     19  
2                 0      3  
3                 0     10  
4                 0      1  


In [ ]:
corpus = set(words.words())

def clean_text(string):
  list_ = string.split(" ")
  # convert all to lower case except special case: "I"
  lower_cases = [word.lower() for word in list_ if word != "I"]
  # only include words that appear in the english language
  valid_words = [word for word in lower_cases if word in corpus]
  return valid_words

In [ ]:
data['body'] = data['body'].apply(lambda x: clean_text(x))
print(data.head())

       subreddit                                               body  \
0  gameofthrones  [your, submission, been, automatically, remove...   
1            aww  [dont, squeeze, her, with, you, massive, you, ...   
2         gaming  [pretty, well, known, and, it, was, a, product...   
3           news  [you, know, we, have, against, that, currently...   
4       politics  [there, is, a, difference, between, gentle, su...   

   controversiality  score  
0                 0      1  
1                 0     19  
2                 0      3  
3                 0     10  
4                 0      1  


In [ ]:
all_dataset_words = []
def compile_all_words(word_list):
  all_dataset_words.extend(word_list)
  return word_list

data['body'] = data['body'].apply(compile_all_words)

all_unique_words = set(all_dataset_words)

In [ ]:
num_unique_words = len(all_unique_words)
num_unique_words

30857

In [ ]:
# create mapping of words to their numbers and vice versa
string_to_int = {s: i for i, s in enumerate(all_unique_words)}
int_to_string = {i: s for s, i in string_to_int.items()}

In [ ]:
# build training data and labels

block_size = 3  # trigram model
X, y = [], []

for index, row in data.iterrows():
  word_list = row['body']
  for i in range(len(word_list) - block_size + 1):
    context = word_list[i:i + block_size - 1]  # context (2 words for trigram)
    target = word_list[i + block_size - 1]  # target (3rd word in trigram)

    # Convert context and target to integer representation
    context_indices = [string_to_int[word] for word in context]
    target_index = string_to_int[target]

    X.append(context_indices)
    y.append(target_index)

In [ ]:
X = torch.tensor(X)
y = torch.tensor(y)

In [ ]:
X.shape, y.shape

(torch.Size([17779504, 2]), torch.Size([17779504]))

In [ ]:
# create embedding tensor -> 90 unique words will be embedded each into 10 values
C = torch.randn((num_unique_words, 10))

In [ ]:
C.shape

torch.Size([30857, 10])

In [ ]:
# lookup of embeddings for every context tensor in X
emb = C[X]
emb.shape

torch.Size([17779504, 2, 10])

## Split into Training and Testing ##

In [ ]:
n_samples = len(X)
reduced_size = n_samples // 1000  # Reduce data size
indices = torch.randperm(n_samples)[:reduced_size]
X_reduced = X[indices]
y_reduced = y[indices]


X_train, X_test, y_train, y_test = train_test_split(
    X_reduced, y_reduced, test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

print(f"Training data: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Testing data: X_test={X_val.shape}, y_test={y_val.shape}")
print(f"Testing data: X_test={X_test.shape}, y_test={y_test.shape}")

Training data: X_train=torch.Size([1137888, 2]), y_train=torch.Size([1137888])
Testing data: X_test=torch.Size([284472, 2]), y_test=torch.Size([284472])
Testing data: X_test=torch.Size([355590, 2]), y_test=torch.Size([355590])


## Prepare Device, Move Data to Device ##

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Currently using {device}")

Currently using cuda


In [ ]:
X_train = X_train.to(device)
X_val = X_val.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_val = y_val.to(device)
y_test = y_test.to(device)

In [ ]:
X_train.shape, X_val.shape, X_test.shape

(torch.Size([1137888, 2]), torch.Size([284472, 2]), torch.Size([355590, 2]))

In [ ]:
y_train.shape, y_val.shape, y_test.shape

(torch.Size([1137888]), torch.Size([284472]), torch.Size([355590]))

## Create Trigram Model ##

In [ ]:
class TrigramModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim, block_size, hidden_size):
    super().__init__()
    self.C = nn.Embedding(vocab_size, embedding_dim)  # Embedding layer
    self.linear_stack = nn.Sequential(
      nn.Linear(block_size * embedding_dim, hidden_size),
      nn.Tanh(),
      nn.Linear(hidden_size, hidden_size),
      nn.Tanh(),
      nn.Linear(hidden_size, vocab_size)  # Output layer
    )

  def forward(self, X):
    print(X.shape)
    emb = self.C(X)  # Convert indices to embeddings
    print(f"emb.shape before view: {emb.shape}")
    emb = emb.view(emb.shape[0], -1)
    print(f"emb shape after view: {emb.shape}")
    logits = self.linear_stack(emb)  # Pass through the linear stack
    return logits

In [ ]:
model = TrigramModel(num_unique_words, 10, 2, 300)
model

TrigramModel(
  (C): Embedding(30857, 10)
  (linear_stack): Sequential(
    (0): Linear(in_features=20, out_features=300, bias=True)
    (1): Tanh()
    (2): Linear(in_features=300, out_features=300, bias=True)
    (3): Tanh()
    (4): Linear(in_features=300, out_features=30857, bias=True)
  )
)

## Model Training ##

In [ ]:
torch.manual_seed(1)

# Hyperparameters
embedding_dim = 10
block_size = 2  # Trigram context size
hidden_size = 300
vocab_size = num_unique_words
epochs = 1000
lr = 0.01

def train_model(embedding_dim, block_size, hidden_size, vocab_size, epochs=1000, lr=0.01):
  # Initialize model, loss function, and optimizer
  model = TrigramModel(vocab_size, embedding_dim, block_size, hidden_size)
  model.to(device)
  loss_fun = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

  # Track losses
  epoch_count = []
  loss_values = []
  test_loss_values = []

  # Training loop
  for epoch in range(epochs):
    # Training phase
    model.train()
    y_pred = model(X_train)
    loss = loss_fun(y_pred, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation phase
    model.eval()
    with torch.no_grad():
      test_pred = model(X_val)
      test_loss = loss_fun(test_pred, y_val)

    # Log metrics
    if epoch % 50 == 0:
      epoch_count.append(epoch)
      loss_values.append(loss.item())
      test_loss_values.append(test_loss.item())
      print(f"Epoch: {epoch} | Loss: {loss.item():.4f} | Test Loss: {test_loss.item():.4f}")

  return

In [ ]:
train_model(10, 2, 300, num_unique_words)

torch.Size([1137888, 2])
emb.shape before view: torch.Size([1137888, 2, 10])
emb shape after view: torch.Size([1137888, 20])


OutOfMemoryError: CUDA out of memory. Tried to allocate 130.80 GiB. GPU 0 has a total capacity of 14.75 GiB of which 3.00 GiB is free. Process 3024 has 11.75 GiB memory in use. Of the allocated memory 11.57 GiB is allocated by PyTorch, and 60.00 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)